### SVM - iris

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()
x = iris.data
y = iris.target

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

In [5]:
from sklearn.svm import SVC
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))

0.9333333333333333


In [17]:
from sklearn.model_selection import GridSearchCV

params = {
    'gamma' : [0.1, 0.01, 0.001],
    'C' : [1, 10, 100]
}

grid_cv = GridSearchCV(clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConve

C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConve

GridSearchCV 최고 평균 정확도 수치:0.9461
GridSearchCV 최적 하이퍼 파라미터: {'C': 10, 'gamma': 0.01}


In [12]:
best_clf = grid_cv.best_estimator_
pred = best_clf.predict(x_test)
print(accuracy_score(y_test, pred))

0.9466666666666667


### SVM - human

In [14]:
import pandas as pd

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                 columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x: x[0]+'_'+str(x[1])
                                                                                              if x[1] > 0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset():
    feature_name_df = pd.read_csv('./datasets/human_activity/features.txt', sep='\s+', header=None, names=['column_index', 'column_name'])
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    x_train = pd.read_csv('./datasets/human_activity/train/X_train.txt', sep='\s+', names=feature_name)
    x_test = pd.read_csv('./datasets/human_activity/test/X_test.txt', sep='\s+', names=feature_name)
    
    y_train = pd.read_csv('./datasets/human_activity/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('./datasets/human_activity/test/y_test.txt', sep='\s+', header=None, names=['action'])
    
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = get_human_dataset()

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

params = {
    'n_estimators' : [100],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(x_train, y_train)

C:\Users\Hong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]})

In [23]:
rf_clf_1 = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=8, min_samples_split=8, random_state=0)

rf_clf_1.fit(x_train, y_train)
pred = rf_clf_1.predict(x_test)

C:\Users\Hong\AppData\Local\Temp\ipykernel_10952\3130665537.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf_1.fit(x_train, y_train)


In [24]:
ftr_importances_values = rf_clf_1.feature_importances_

In [25]:
ftr_importances = pd.Series(ftr_importances_values, index=x_train.columns)
ftr_top10 = ftr_importances.sort_values(ascending=False)[:10]

In [28]:
top10_index = ftr_top10.index
x_train_10 = x_train.loc[:,top10_index]
x_test_10 = x_test.loc[:,top10_index]

In [29]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


clf = SVC(gamma=0.001,C=100)
clf.fit(x_train_10,y_train)
pred = clf.predict(x_test_10)
acc = accuracy_score(y_test,pred)

print(acc)

pram = {'C':[1,10,100],
       'gamma':[0.1,0.01,0.001]}
grid = GridSearchCV(clf,param_grid=pram,cv=2,scoring="accuracy")
grid.fit(x_train_10,y_train)
best_clf = grid.best_estimator_
best_pred = best_clf.predict(x_test_10)
best_acc = accuracy_score(y_test,best_pred)

print(best_acc)

0.5225653206650831
0.6386155412283678
